In [4]:
import networkx as nx
import plotly.graph_objects as go
import itertools
from collections import Counter

countries = [
    "Afghanistan", "Albania", "Algeria", "Andorra", "Angola", "Antigua and Barbuda",
    "Argentina", "Armenia", "Australia", "Austria", "Azerbaijan", "Bahamas",
    "Bahrain", "Bangladesh", "Barbados", "Belarus", "Belgium", "Belize", "Benin",
    "Bhutan", "Bolivia", "Bosnia and Herzegovina", "Botswana", "Brazil", "Brunei",
    "Bulgaria", "Burkina Faso", "Burundi", "Cambodia", "Cameroon", "Canada",
    "Cape Verde", "Central African Republic", "Chad", "Chile", "China", "Colombia",
    "Comoros", "Congo", "Costa Rica", "Côte d'Ivoire", "Croatia", "Cuba", "Cyprus",
    "Czech Republic", "North Korea", "Democratic Republic of the Congo", "Denmark",
    "Djibouti", "Dominica", "Dominican Republic", "Ecuador", "Egypt", "El Salvador",
    "Equatorial Guinea", "Eritrea", "Estonia", "Eswatini", "Ethiopia", "Fiji",
    "Finland", "France", "Gabon", "Gambia", "Georgia", "Germany", "Ghana", "Greece",
    "Grenada", "Guatemala", "Guinea", "Guinea-Bissau", "Guyana", "Haiti", "Honduras",
    "Hungary", "Iceland", "India", "Indonesia", "Iran", "Iraq", "Ireland", "Israel",
    "Italy", "Jamaica", "Japan", "Jordan", "Kazakhstan", "Kenya", "Kiribati",
    "Kuwait", "Kyrgyzstan", "Laos", "Latvia", "Lebanon", "Lesotho", "Liberia",
    "Libya", "Liechtenstein", "Lithuania", "Luxembourg", "Madagascar", "Malawi",
    "Malaysia", "Maldives", "Mali", "Malta", "Marshall Islands", "Mauritania",
    "Mauritius", "Mexico", "Micronesia", "Monaco", "Mongolia", "Montenegro",
    "Morocco", "Mozambique", "Myanmar", "Namibia", "Nauru", "Nepal", "Netherlands",
    "New Zealand", "Nicaragua", "Niger", "Nigeria", "North Macedonia", "Norway",
    "Oman", "Pakistan", "Palau", "Panama", "Papua New Guinea", "Paraguay", "Peru",
    "Philippines", "Poland", "Portugal", "Qatar", "South Korea", "Moldova",
    "Romania", "Russia", "Rwanda", "Saint Kitts and Nevis", "Saint Lucia",
    "Saint Vincent and the Grenadines", "Samoa", "San Marino", "São Tomé and Príncipe",
    "Saudi Arabia", "Senegal", "Serbia", "Seychelles", "Sierra Leone", "Singapore",
    "Slovakia", "Slovenia", "Solomon Islands", "Somalia", "South Africa",
    "South Sudan", "Spain", "Sri Lanka", "Sudan", "Suriname", "Sweden",
    "Switzerland", "Syria", "Tajikistan", "Tanzania", "Thailand", "Timor-Leste",
    "Togo", "Tonga", "Trinidad and Tobago", "Tunisia", "Turkey", "Turkmenistan",
    "Tuvalu", "Uganda", "Ukraine", "United Arab Emirates", "United Kingdom",
    "United States", "Uruguay", "Uzbekistan", "Vanuatu", "Venezuela", "Vietnam",
    "Yemen", "Zambia", "Zimbabwe"
]

atlas_graph = nx.DiGraph()
atlas_graph.add_nodes_from(countries)

edges = [
    (c1, c2) for c1, c2 in itertools.product(countries, repeat=2)
    if c1 != c2 and c1.lower()[-1] == c2.lower()[0]
]
atlas_graph.add_edges_from(edges)

def create_clean_interactive_graph(G):
    pos = nx.kamada_kawai_layout(G)

    edge_x, edge_y = [], []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])

    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=0.4, color='#888'),
        hoverinfo='none',
        mode='lines',
        visible='legendonly',
        name='Connections'
    )

    node_x, node_y, node_hovertext, node_color, node_size = [], [], [], [], []
    for node in G.nodes():
        x, y = pos[node]
        in_degree = G.in_degree(node)
        out_degree = G.out_degree(node)

        node_x.append(x)
        node_y.append(y)

        hovertext = (
            f"<b>{node}</b><br>"
            f"Options From Here: {out_degree}<br>"
            f"Paths To Here: {in_degree}<br>"
            f"Strategic Value: {out_degree - in_degree}"
        )
        node_hovertext.append(hovertext)
        node_color.append(out_degree - in_degree)
        node_size.append(5 + (in_degree + out_degree) / 2)

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers',
        hoverinfo='text',
        text=node_hovertext,
        marker=dict(
            showscale=True,
            colorscale='RdYlGn',
            color=node_color,
            size=node_size,
            colorbar=dict(
                thickness=15,
                title='Strategic Value',
                xanchor='left',
                titleside='right'
            ),
            line_width=1,
            line_color='black'
        ),
        name='Countries'
    )

    fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='Atlas Game: Strategic Map of Countries',
                titlefont_size=20,
                showlegend=True,
                legend_title_text='Toggle View',
                legend=dict(
                    x=0.01,
                    y=0.99,
                    xanchor='left',
                    yanchor='top'
                ),
                hovermode='closest',
                margin=dict(b=5, l=5, r=5, t=40),
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                plot_bgcolor='white',
                height=700,
                width=1000
            )
    )

    fig.show()

create_clean_interactive_graph(atlas_graph)

In [7]:
import networkx as nx
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from collections import Counter

def perform_graph_analysis(G):
    analysis = {}
    analysis['in_degrees'] = dict(G.in_degree)
    analysis['out_degrees'] = dict(G.out_degree)
    analysis['dead_ends'] = [n for n, d in analysis['out_degrees'].items() if d == 0]

    analysis['pagerank'] = nx.pagerank(G)
    analysis['betweenness'] = nx.betweenness_centrality(G)

    start_letters = Counter(c[0].lower() for c in G.nodes())
    end_letters = Counter(c[-1].lower() for c in G.nodes())

    letter_advantage = {}
    all_letters = sorted(list(set(start_letters.keys()) | set(end_letters.keys())))
    for letter in all_letters:
        advantage = start_letters.get(letter, 0) - end_letters.get(letter, 0)
        letter_advantage[letter] = advantage

    analysis['letter_advantage'] = letter_advantage
    analysis['start_letters'] = start_letters
    analysis['end_letters'] = end_letters
    analysis['all_letters'] = all_letters

    return analysis

def create_analysis_dashboard(results):
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            "Options After Naming (Out-Degree)",
            "How Often a Country is an Option (In-Degree)",
            "Starting vs. Ending Letter Frequency",
            "Strategic Letter Advantage"
        ),
        vertical_spacing=0.15,
        horizontal_spacing=0.05
    )

    fig.add_trace(go.Histogram(x=list(results['out_degrees'].values()), name='Out-Degree'), row=1, col=1)
    fig.add_trace(go.Histogram(x=list(results['in_degrees'].values()), name='In-Degree'), row=1, col=2)

    start_counts = [results['start_letters'].get(l, 0) for l in results['all_letters']]
    end_counts = [results['end_letters'].get(l, 0) for l in results['all_letters']]
    fig.add_trace(go.Bar(x=[l.upper() for l in results['all_letters']], y=start_counts, name='Starts With'), row=2, col=1)
    fig.add_trace(go.Bar(x=[l.upper() for l in results['all_letters']], y=end_counts, name='Ends With'), row=2, col=1)

    advantages = list(results['letter_advantage'].values())
    colors = ['green' if adv > 0 else 'red' if adv < 0 else 'grey' for adv in advantages]
    fig.add_trace(go.Bar(x=[l.upper() for l in results['all_letters']], y=advantages, marker_color=colors, name='Advantage'), row=2, col=2)

    fig.update_layout(
        height=800, width=1200,
        title_text="Atlas Game: Strategic Analysis Dashboard",
        barmode='group', showlegend=False
    )
    fig.update_xaxes(title_text="Degree Count", row=1, col=1)
    fig.update_xaxes(title_text="Degree Count", row=1, col=2)
    fig.update_xaxes(title_text="Letter", row=2, col=1)
    fig.update_xaxes(title_text="Letter", row=2, col=2)
    fig.show()

class AtlasGameAdvisor:
    def __init__(self, graph, analysis_results):
        self.graph = graph
        self.analysis = analysis_results
        self.used_countries = set()
        self.country_scores = self._calculate_scores()

    def _calculate_scores(self):
        scores = {}
        for country in self.graph.nodes():
            out_deg = self.analysis['out_degrees'].get(country, 0)
            pr = self.analysis['pagerank'].get(country, 0)
            betweenness = self.analysis['betweenness'].get(country, 0)

            score = (out_deg * 2) + (pr * 500) + (betweenness * 200)
            if out_deg == 0:
                score -= 100
            scores[country] = score
        return scores

    def get_valid_moves(self, last_country):
        if last_country not in self.graph: return []
        return [
            successor for successor in self.graph.successors(last_country)
            if successor not in self.used_countries
        ]

    def recommend_moves(self, last_country):
        valid_moves = self.get_valid_moves(last_country)
        if not valid_moves:
            return "Opponent is trapped! You win!"

        scored_moves = sorted(
            [(move, self.country_scores.get(move, 0)) for move in valid_moves],
            key=lambda x: x[1],
            reverse=True
        )
        return scored_moves

    def mark_used(self, country):
        self.used_countries.add(country)

    def reset(self):
        self.used_countries.clear()

def start_interactive_session(advisor, graph_nodes):
    print("\n" + "="*60)
    print("ATLAS GAME STRATEGY ADVISOR")
    print("="*60)
    print("Enter opponent's move to get strategic advice.")
    print("Commands: 'reset', 'used', 'stats', 'quit'")

    while True:
        user_input = input("\n> Opponent played: ").strip()

        if user_input.lower() == 'quit':
            break
        elif user_input.lower() == 'reset':
            advisor.reset()
            print("Game has been reset.")
            continue
        elif user_input.lower() == 'used':
            if not advisor.used_countries:
                print("No countries used yet.")
            else:
                print("Used countries:", sorted(list(advisor.used_countries)))
            continue
        elif user_input.lower() == 'stats':
            remaining = advisor.graph.number_of_nodes() - len(advisor.used_countries)
            print(f"Countries used: {len(advisor.used_countries)}. Remaining: {remaining}.")
            continue

        matches = [c for c in graph_nodes if c.lower().startswith(user_input.lower())]
        if not matches:
            print(f"Sorry, couldn't find a country starting with '{user_input}'.")
            continue

        selected_country = matches[0]
        if selected_country in advisor.used_countries:
            print(f"'{selected_country}' has already been used.")
            continue

        advisor.mark_used(selected_country)
        print(f"Opponent's move '{selected_country}' marked as used.")

        recommendations = advisor.recommend_moves(selected_country)

        if isinstance(recommendations, str):
            print(f"\n{recommendations}")
            break

        print("\n--- Recommended Moves (Best to Worst) ---")
        for i, (move, score) in enumerate(recommendations[:5]):
            dead_end_tag = " (WINNING TRAP!)" if advisor.analysis['out_degrees'].get(move, 0) == 0 else ""
            print(f"{i+1}. {move}{dead_end_tag}")

        if recommendations:
            best_move = recommendations[0][0]
            choice = input(f"Execute best move '{best_move}'? (y/n): ").strip().lower()
            if choice == 'y':
                advisor.mark_used(best_move)
                print(f"'{best_move}' marked as used. Your turn is over.")


analysis_results = perform_graph_analysis(atlas_graph)

create_analysis_dashboard(analysis_results)

advisor = AtlasGameAdvisor(atlas_graph, analysis_results)



In [8]:
start_interactive_session(advisor, list(atlas_graph.nodes()))


ATLAS GAME STRATEGY ADVISOR
Enter opponent's move to get strategic advice.
Commands: 'reset', 'used', 'stats', 'quit'

> Opponent played: Sudan
Opponent's move 'Sudan' marked as used.

--- Recommended Moves (Best to Worst) ---
1. Netherlands
2. Nauru
3. New Zealand
4. Nepal
5. North Korea
Execute best move 'Netherlands'? (y/n): y
'Netherlands' marked as used. Your turn is over.

> Opponent played: South Korea
Opponent's move 'South Korea' marked as used.

--- Recommended Moves (Best to Worst) ---
1. Afghanistan
2. Azerbaijan
3. Albania
4. Algeria
5. Andorra
Execute best move 'Afghanistan'? (y/n): y
'Afghanistan' marked as used. Your turn is over.

> Opponent played: Nepal
Opponent's move 'Nepal' marked as used.

--- Recommended Moves (Best to Worst) ---
1. Laos
2. Luxembourg
3. Lebanon
4. Liechtenstein
5. Latvia
Execute best move 'Laos'? (y/n): y
'Laos' marked as used. Your turn is over.

> Opponent played: South Africa
Opponent's move 'South Africa' marked as used.

--- Recommended M